## Preprocessing and Feature Extraction

In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

Sentiment Analysis:

Steps taken:
- Converting emojis to text
- Lowercase
- Remove all the hashtagged words
- Removing Punctuation
- Tokenization
- Stopword filtering
- Stemming

In [5]:
df = pd.read_csv("/Users/harveymiller/Documents/GitHub/text-success/Data/tweets_ukraine_monthly.csv")

Creating a new column so that we can see the adjusted tweet and original versiom

In [ ]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

In [11]:
df.head()

,id,date,user,user_followers,raw_content,rendered_content,Adjusted Tweet,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers
0,1486661338390831105,2022-01-27 11:24:18+00:00,WorldToBe1,10769,#Russia-#Ukraine debate sparks fiery exchange ...,#Russia-#Ukraine debate sparks fiery exchange ...,#Russia-#Ukraine debate sparks fiery exchange ...,0,0,0,0,"['Russia', 'Ukraine', 'CNN', 'USA', 'EU', 'NAT...",en,NaN,NaN
1,1486105614803775490,2022-01-25 22:36:03+00:00,embeegle,2246,Can you say pipeline? A larger cut coming to ...,Can you say pipeline? A larger cut coming to ...,Can you say pipeline? A larger cut coming to ...,1,1,0,0,"['PutinsPuppet', 'ukrainewar']",en,NaN,NaN
2,1486056169013661697,2022-01-25 19:19:34+00:00,Vlad_Mykhnenko,1068,Foreign exchange markets somehow are not betti...,Foreign exchange markets somehow are not betti...,Foreign exchange markets somehow are not betti...,1,0,0,0,"['russianinvasion', 'ukrainewar']",en,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN
3,1486019310069989376,2022-01-25 16:53:07+00:00,RaVe_74,5309,#borisjohnson's expertise in foreign affairs -...,#borisjohnson's expertise in foreign affairs -...,#borisjohnson's expertise in foreign affairs -...,3,1,0,0,"['borisjohnson', 'freenazanin', 'BrexitShamble...",en,NaN,NaN
4,1485989417084985347,2022-01-25 14:54:20+00:00,miamicool,1035,"Seems that #ukrainewar just became that ""line ...","Seems that #ukrainewar just became that ""line ...","Seems that #ukrainewar just became that ""line ...",0,0,0,0,['ukrainewar'],en,NaN,NaN


Converting emojis to text

In [12]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

Changing all text to lowercase

In [19]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing hashtagged words

Removing all Punctuation

In [20]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [21]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [23]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [24]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

In [25]:
df.head()

,id,date,user,user_followers,raw_content,rendered_content,Adjusted Tweet,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers
0,1486661338390831105,2022-01-27 11:24:18+00:00,WorldToBe1,10769,#Russia-#Ukraine debate sparks fiery exchange ...,#Russia-#Ukraine debate sparks fiery exchange ...,"[russiaukraine, debate, sparks, fiery, exchang...",0,0,0,0,"['Russia', 'Ukraine', 'CNN', 'USA', 'EU', 'NAT...",en,NaN,NaN
1,1486105614803775490,2022-01-25 22:36:03+00:00,embeegle,2246,Can you say pipeline? A larger cut coming to ...,Can you say pipeline? A larger cut coming to ...,"[say, pipeline, larger, cut, coming, putinspup...",1,1,0,0,"['PutinsPuppet', 'ukrainewar']",en,NaN,NaN
2,1486056169013661697,2022-01-25 19:19:34+00:00,Vlad_Mykhnenko,1068,Foreign exchange markets somehow are not betti...,Foreign exchange markets somehow are not betti...,"[foreign, exchange, markets, somehow, betting,...",1,0,0,0,"['russianinvasion', 'ukrainewar']",en,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN
3,1486019310069989376,2022-01-25 16:53:07+00:00,RaVe_74,5309,#borisjohnson's expertise in foreign affairs -...,#borisjohnson's expertise in foreign affairs -...,"[borisjohnsons, expertise, foreign, affairs, t...",3,1,0,0,"['borisjohnson', 'freenazanin', 'BrexitShamble...",en,NaN,NaN
4,1485989417084985347,2022-01-25 14:54:20+00:00,miamicool,1035,"Seems that #ukrainewar just became that ""line ...","Seems that #ukrainewar just became that ""line ...","[seems, ukrainewar, became, line, sand, whole,...",0,0,0,0,['ukrainewar'],en,NaN,NaN
